Installing Pyspark

In [36]:
!pip install pyspark databricks

Importing Libraries


In [51]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
from pyspark.sql.functions import *
from databricks import *

Creating Dataframe

In [7]:
spark = SparkSession.builder.appName('pyspark-project').getOrCreate()

df = spark.read.load('/content/sample_data/googleplaystore.csv',format='csv',sep=',',header='true',inferschema='true',escape='"')

df.show()

+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|      Last Updated|       Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|    10,000+|Free|    0|      Everyone|        Art & Design|   January 7, 2018|             1.0.0|4.0.3 and up|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967| 14M|   500,000+|Free|    0|      Everyone|Art & Design;Pret...|  January 15, 2018|             2.0.0|4.0.3 and up|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|8.7M| 5,000,000+|Free|    0|      Everyone|        Art & Design|    August 1, 2018|             1.2.4|4.0.3 

In [9]:
#Getting total count

df.count()

10841

In [10]:
#First Row

df.show(1)

+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
|                 App|      Category|Rating|Reviews|Size|Installs|Type|Price|Content Rating|      Genres|   Last Updated|Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M| 10,000+|Free|    0|      Everyone|Art & Design|January 7, 2018|      1.0.0|4.0.3 and up|
+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
only showing top 1 row



Check Schema

In [11]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



Data Cleaning

In [13]:
#dropping the columns which are not important

df = df.drop('size','Content Rating','Last Updated','Current Ver','Android Ver')
df.show(2)

+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 10,000+|Free|    0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|500,000+|Free|    0|Art & Design;Pret...|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
only showing top 2 rows



In [15]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Genres: string (nullable = true)



In [52]:
#Changing schemas into proper format

from pyspark.sql.functions import regexp_replace, col

regex_pattern = r'^\d*\.?\d+$'

df = df.withColumn('Reviews',col('Reviews').cast(IntegerType()))\
.withColumn('Installs',regexp_replace(col('Installs'),"[^0-9]",""))\
.withColumn('Installs',col('Installs').cast(IntegerType()))\
.withColumn('Price',regexp_replace(col('Price'),"[$]",""))\
.withColumn('Price',col('Price').cast(IntegerType()))\
.filter(col('Rating').rlike(regex_pattern))\
.withColumn('Rating',col('Rating').cast(DoubleType())).filter(col('Rating').isNotNull())

In [53]:
df.show()

+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|   10000|Free|    0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|  500000|Free|    0|Art & Design;Pret...|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510| 5000000|Free|    0|        Art & Design|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5| 215644|50000000|Free|    0|        Art & Design|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    967|  100000|Free|    0|Art & Design;Crea...|
|Paper flowers ins...|ART_AND_DESIGN|   4.4|    167|   50000|Free|    0|        Art & Design|
|Smoke Effect Phot...|ART_AND_DESIGN|   3.8|    178|   50000|Free|    0|        Art & Design|
|    Infinite Painter|ART_AND_DESIGN|   4.1|  36815| 1000000

In [54]:
#Verifying schema changes

df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Installs: integer (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Genres: string (nullable = true)



In [21]:
df.select("*").show()

+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|   10000|Free|    0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|  500000|Free|    0|Art & Design;Pret...|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510| 5000000|Free|    0|        Art & Design|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5| 215644|50000000|Free|    0|        Art & Design|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    967|  100000|Free|    0|Art & Design;Crea...|
|Paper flowers ins...|ART_AND_DESIGN|   4.4|    167|   50000|Free|    0|        Art & Design|
|Smoke Effect Phot...|ART_AND_DESIGN|   3.8|    178|   50000|Free|    0|        Art & Design|
|    Infinite Painter|ART_AND_DESIGN|   4.1|  36815| 1000000

Top most reviewed Apps

In [30]:
top_reviewed_apps = df.groupBy('App')\
.agg(sum("Reviews").alias("total_reviews"))\
.orderBy(col("total_reviews").desc())

top_reviewed_apps.show(10,truncate=False) #Top 10 reviewed apps

+----------------------------------------+-------------+
|App                                     |total_reviews|
+----------------------------------------+-------------+
|Instagram                               |266241989    |
|WhatsApp Messenger                      |207348304    |
|Clash of Clans                          |179558781    |
|Messenger – Text and Video Chat for Free|169932272    |
|Subway Surfers                          |166331958    |
|Candy Crush Saga                        |156993136    |
|Facebook                                |156286514    |
|8 Ball Pool                             |99386198     |
|Clash Royale                            |92530298     |
|Snapchat                                |68045010     |
+----------------------------------------+-------------+
only showing top 10 rows



Top 10 installed Apps

In [41]:
top_installed_apps = df.groupBy('App','Type')\
.agg(sum('Installs').alias('total_installs'))\
.orderBy(col("total_installs").desc())

top_installed_apps.show(10, truncate = False) #top 10 installed apps

+------------------+----+--------------+
|App               |Type|total_installs|
+------------------+----+--------------+
|Subway Surfers    |Free|6000000000    |
|Instagram         |Free|4000000000    |
|Google Drive      |Free|4000000000    |
|Hangouts          |Free|4000000000    |
|Google Photos     |Free|4000000000    |
|Google News       |Free|4000000000    |
|Candy Crush Saga  |Free|3500000000    |
|WhatsApp Messenger|Free|3000000000    |
|Gmail             |Free|3000000000    |
|Temple Run 2      |Free|3000000000    |
+------------------+----+--------------+
only showing top 10 rows



Category wise distribution of installed apps

In [45]:
category_dist_apps = df.groupBy('Category')\
.agg(sum('Installs').alias('total_installs'))\
.orderBy(col('total_installs').desc())

#category_dist_apps.show(n = category_dist_apps.count(),truncate = False)
category_dist_apps.show(10,truncate = False) #top 10 categories

+------------------+--------------+
|Category          |total_installs|
+------------------+--------------+
|GAME              |35086024415   |
|COMMUNICATION     |32647276251   |
|PRODUCTIVITY      |14176091369   |
|SOCIAL            |14069867902   |
|TOOLS             |11452771915   |
|FAMILY            |10258263505   |
|PHOTOGRAPHY       |10088247655   |
|NEWS_AND_MAGAZINES|7496317760    |
|TRAVEL_AND_LOCAL  |6868887146    |
|VIDEO_PLAYERS     |6222002720    |
+------------------+--------------+
only showing top 10 rows



Expensive Apps

In [49]:
expensive_apps = df.groupBy('App')\
.agg(sum('Price').alias('Price'))\
.orderBy(col('Price').desc())

expensive_apps.show(10, truncate = False)

+-------------------------------------+-----+
|App                                  |Price|
+-------------------------------------+-----+
|I'm Rich - Trump Edition             |400  |
|most expensive app (H)               |399  |
|I am Rich                            |399  |
|I AM RICH PRO PLUS                   |399  |
|I am rich(premium)                   |399  |
|I am Rich Plus                       |399  |
|I'm Rich/Eu sou Rico/أنا غني/我很有錢|399  |
|I Am Rich Premium                    |399  |
|I Am Rich Pro                        |399  |
|I am Rich!                           |399  |
+-------------------------------------+-----+
only showing top 10 rows



Top rating in paid Apps

In [56]:
filtered_df = df.filter(col('Type')=='Paid')

top_rated_paid = filtered_df.groupBy('App')\
.agg(avg('Rating').alias('average_rating'))\
.orderBy(col('average_rating').desc())

top_rated_paid.show(30, truncate = False)

+--------------------------------------------------+--------------+
|App                                               |average_rating|
+--------------------------------------------------+--------------+
|Morse Player                                      |5.0           |
|AJ Gray Dark Icon Pack                            |5.0           |
|Hey AJ! It's Bedtime!                             |5.0           |
|AP Art History Flashcards                         |5.0           |
|P-Home for KLWP                                   |5.0           |
|Easy Hotspot Ad Free                              |5.0           |
|DL Image Manager                                  |5.0           |
|FHR 5-Tier 2.0                                    |5.0           |
|Super Hearing Secret Voices Recorder PRO          |5.0           |
|AJ Blue Icon Pack                                 |5.0           |
|Mu.F.O.                                           |5.0           |
|30WPM Amateur ham radio Koch CW Morse code trai